# Atividade 1 — Aprendizado Semi-supervisionado (Auto-Treinamento) com SpamBase

## Objetivo
Melhorar um classificador de spam usando poucos dados rotulados (10%) + pseudo-rotulagem dos dados não rotulados (90%).

## Dataset
SpamBase (OpenML id=44) - 57 características numéricas; classificação binária spam vs. não-spam.

## 1. Importar Bibliotecas

In [ ]:
# Importações necessárias
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 2. Carregar e Preparar Dados

In [ ]:
# Carregar dataset SpamBase
print("Carregando dataset SpamBase...")
data = fetch_openml(data_id=44, as_frame=True, parser='auto')

X = data.data
y = data.target.astype(int)  # 1=spam, 0=ham (não-spam)

print(f"Shape dos dados: {X.shape}")
print(f"Distribuição de classes: \n{pd.Series(y).value_counts()}")
print(f"\nPrimeiras linhas:")
print(X.head())

## 3. Dividir em Conjunto Rotulado (10%) e Não Rotulado (90%)

In [ ]:
# Primeiro, separar conjunto de teste (20% do total)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Agora dividir o conjunto de treino em rotulado (10%) e não rotulado (90%)
X_lab, X_unlab, y_lab, y_unlab = train_test_split(
    X_train_full, y_train_full, test_size=0.9, stratify=y_train_full, random_state=42
)

print(f"Dados rotulados: {X_lab.shape[0]} amostras")
print(f"Dados não rotulados: {X_unlab.shape[0]} amostras")
print(f"Dados de teste: {X_test.shape[0]} amostras")

# Normalizar os dados
scaler = StandardScaler()
X_lab_scaled = scaler.fit_transform(X_lab)
X_unlab_scaled = scaler.transform(X_unlab)
X_test_scaled = scaler.transform(X_test)

## 4. Baseline Supervisionado (Apenas 10% dos Dados Rotulados)

In [ ]:
# Treinar modelo baseline com apenas dados rotulados
print("Treinando modelo baseline...")
baseline_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
baseline_model.fit(X_lab_scaled, y_lab)

# Avaliar no conjunto de teste
y_pred_baseline = baseline_model.predict(X_test_scaled)

# Métricas
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
baseline_precision = precision_score(y_test, y_pred_baseline)
baseline_recall = recall_score(y_test, y_pred_baseline)
baseline_f1 = f1_score(y_test, y_pred_baseline)

print("\n=== RESULTADOS BASELINE ===")
print(f"Acurácia: {baseline_accuracy:.4f}")
print(f"Precisão: {baseline_precision:.4f}")
print(f"Recall: {baseline_recall:.4f}")
print(f"F1-Score: {baseline_f1:.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_baseline, target_names=['Ham', 'Spam']))

## 5. Auto-Treinamento (Self-Training) com Pseudo-Rotulagem

In [ ]:
# Implementação do algoritmo de Auto-Treinamento
def self_training(X_labeled, y_labeled, X_unlabeled, threshold=0.95, max_iterations=10, batch_size=50):
    """
    Algoritmo de Auto-Treinamento (Self-Training)
    
    Parâmetros:
    - X_labeled: dados rotulados
    - y_labeled: rótulos conhecidos
    - X_unlabeled: dados não rotulados
    - threshold: limite de confiança para pseudo-rotulagem
    - max_iterations: número máximo de iterações
    - batch_size: quantidade de amostras a adicionar por iteração
    """
    
    # Copiar dados para não modificar os originais
    X_train = X_labeled.copy()
    y_train = y_labeled.copy()
    X_pool = X_unlabeled.copy()
    
    history = {'iteration': [], 'labeled_samples': [], 'added_samples': []}
    
    for iteration in range(max_iterations):
        if len(X_pool) == 0:
            print(f"Iteração {iteration}: Pool de dados não rotulados vazio.")
            break
            
        # Treinar modelo com dados atuais
        model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)
        
        # Fazer predições no pool não rotulado
        probas = model.predict_proba(X_pool)
        max_probas = np.max(probas, axis=1)
        predictions = model.predict(X_pool)
        
        # Selecionar amostras com alta confiança
        confident_indices = np.where(max_probas >= threshold)[0]
        
        if len(confident_indices) == 0:
            print(f"Iteração {iteration}: Nenhuma amostra com confiança >= {threshold}")
            break
        
        # Limitar o número de amostras adicionadas
        if len(confident_indices) > batch_size:
            # Selecionar as mais confiantes
            top_indices = np.argsort(max_probas[confident_indices])[-batch_size:]
            confident_indices = confident_indices[top_indices]
        
        # Adicionar ao conjunto de treino
        X_to_add = X_pool[confident_indices]
        y_to_add = predictions[confident_indices]
        
        X_train = np.vstack([X_train, X_to_add])
        y_train = np.concatenate([y_train, y_to_add])
        
        # Remover do pool não rotulado
        X_pool = np.delete(X_pool, confident_indices, axis=0)
        
        # Registrar histórico
        history['iteration'].append(iteration)
        history['labeled_samples'].append(len(y_train))
        history['added_samples'].append(len(confident_indices))
        
        print(f"Iteração {iteration}: Adicionadas {len(confident_indices)} amostras. Total rotulado: {len(y_train)}")
    
    return X_train, y_train, history

# Executar auto-treinamento
print("Iniciando Auto-Treinamento...\n")
X_train_st, y_train_st, history = self_training(
    X_lab_scaled, y_lab, X_unlab_scaled, 
    threshold=0.95, max_iterations=10, batch_size=50
)

In [ ]:
# Visualizar evolução do auto-treinamento
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history['iteration'], history['labeled_samples'], marker='o')
plt.xlabel('Iteração')
plt.ylabel('Total de Amostras Rotuladas')
plt.title('Crescimento do Conjunto de Treinamento')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.bar(history['iteration'], history['added_samples'])
plt.xlabel('Iteração')
plt.ylabel('Amostras Adicionadas')
plt.title('Amostras Pseudo-Rotuladas por Iteração')
plt.grid(True, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Treinar modelo final com dados aumentados
print("\nTreinando modelo final com auto-treinamento...")
st_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
st_model.fit(X_train_st, y_train_st)

# Avaliar no conjunto de teste
y_pred_st = st_model.predict(X_test_scaled)

# Métricas
st_accuracy = accuracy_score(y_test, y_pred_st)
st_precision = precision_score(y_test, y_pred_st)
st_recall = recall_score(y_test, y_pred_st)
st_f1 = f1_score(y_test, y_pred_st)

print("\n=== RESULTADOS AUTO-TREINAMENTO ===")
print(f"Acurácia: {st_accuracy:.4f}")
print(f"Precisão: {st_precision:.4f}")
print(f"Recall: {st_recall:.4f}")
print(f"F1-Score: {st_f1:.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_st, target_names=['Ham', 'Spam']))

## 6. Comparação de Métricas

In [ ]:
# Comparar métricas
metrics_comparison = pd.DataFrame({
    'Baseline (10%)': [baseline_accuracy, baseline_precision, baseline_recall, baseline_f1],
    'Auto-Treinamento': [st_accuracy, st_precision, st_recall, st_f1]
}, index=['Acurácia', 'Precisão', 'Recall', 'F1-Score'])

print("\n=== COMPARAÇÃO DE MÉTRICAS ===")
print(metrics_comparison)
print("\nMelhoria relativa (%)")
improvement = ((metrics_comparison['Auto-Treinamento'] - metrics_comparison['Baseline (10%)']) / 
               metrics_comparison['Baseline (10%)'] * 100)
print(improvement)

In [ ]:
# Gráfico de comparação
metrics_comparison.plot(kind='bar', figsize=(10, 6), rot=0)
plt.title('Comparação: Baseline vs. Auto-Treinamento', fontsize=14, fontweight='bold')
plt.ylabel('Score')
plt.xlabel('Métrica')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Matrizes de Confusão

In [ ]:
# Calcular matrizes de confusão
cm_baseline = confusion_matrix(y_test, y_pred_baseline)
cm_st = confusion_matrix(y_test, y_pred_st)

# Plotar matrizes de confusão lado a lado
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Baseline
sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'], ax=axes[0])
axes[0].set_title('Matriz de Confusão - Baseline (10%)', fontweight='bold')
axes[0].set_ylabel('Rótulo Verdadeiro')
axes[0].set_xlabel('Rótulo Predito')

# Auto-Treinamento
sns.heatmap(cm_st, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'], ax=axes[1])
axes[1].set_title('Matriz de Confusão - Auto-Treinamento', fontweight='bold')
axes[1].set_ylabel('Rótulo Verdadeiro')
axes[1].set_xlabel('Rótulo Predito')

plt.tight_layout()
plt.show()

## 8. Análise de Erros

In [ ]:
# Analisar tipos de erros
print("=== ANÁLISE DE ERROS ===")
print("\nBaseline:")
print(f"  Falsos Positivos (Ham classificado como Spam): {cm_baseline[0, 1]}")
print(f"  Falsos Negativos (Spam classificado como Ham): {cm_baseline[1, 0]}")
print(f"  Total de erros: {cm_baseline[0, 1] + cm_baseline[1, 0]}")

print("\nAuto-Treinamento:")
print(f"  Falsos Positivos (Ham classificado como Spam): {cm_st[0, 1]}")
print(f"  Falsos Negativos (Spam classificado como Ham): {cm_st[1, 0]}")
print(f"  Total de erros: {cm_st[0, 1] + cm_st[1, 0]}")

print("\nRedução de erros:")
error_reduction = (cm_baseline[0, 1] + cm_baseline[1, 0]) - (cm_st[0, 1] + cm_st[1, 0])
print(f"  {error_reduction} erros a menos com auto-treinamento")
print(f"  Redução de {error_reduction / (cm_baseline[0, 1] + cm_baseline[1, 0]) * 100:.1f}%")

## 9. Conclusões

### Observações:

1. **Performance do Baseline**: Com apenas 10% dos dados rotulados, o modelo baseline já consegue uma performance razoável, mas limitada.

2. **Impacto do Auto-Treinamento**: 
   - O auto-treinamento utiliza predições confiantes para expandir o conjunto de treinamento
   - Isso permite ao modelo aprender padrões adicionais dos dados não rotulados
   - Geralmente observamos melhoria em todas as métricas (F1, Precisão, Recall)

3. **Tipos de Erros**:
   - **Falsos Positivos**: Emails legítimos classificados como spam (incômodo para usuário)
   - **Falsos Negativos**: Spam não detectado (risco de segurança)
   - O auto-treinamento tende a reduzir ambos os tipos de erro

4. **Limitações**:
   - O sucesso depende da qualidade das predições iniciais
   - Se o modelo inicial estiver muito enviesado, pode propagar erros
   - O threshold de confiança é crucial para o desempenho

5. **Aplicações Práticas**:
   - Útil quando rotular dados é caro ou demorado
   - Filtros de spam, detecção de fraude, classificação de documentos
   - Cenários onde há muitos dados não rotulados disponíveis